In [2]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from pyimagesearch.preprocessing import ImageToArrayPreprocessor
from pyimagesearch.preprocessing import AspectAwarePreprocessor
from pyimagesearch.datasets import SimpleDatasetLoader
from pyimagesearch.nn.conv import MiniVGGNet
from keras.optimizers import SGD
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os

Using TensorFlow backend.
/data/tangle/pyimagesearch/venv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
imagePaths = list(paths.list_images("flowers-17"))
classNames = [pt.split(os.path.sep)[-2] for pt in imagePaths]
classNames = [str(x) for x in np.unique(classNames)]

In [6]:
len(classNames)

17

In [7]:
aap = AspectAwarePreprocessor(64, 64)
iap = ImageToArrayPreprocessor()
sdl = SimpleDatasetLoader(preprocessors=[aap, iap])
(data, labels) = sdl.load(imagePaths, verbose=500)
data = data.astype("float") / 255.0

[INFO] processed 500/1360
[INFO] processed 1000/1360


In [8]:
data.shape

(1360, 64, 64, 3)

In [9]:
(trainX, testX, trainY, testY) = train_test_split(data, labels,
                                                 test_size=0.25, random_state=42)

In [12]:
trainY = LabelBinarizer().fit_transform(trainY)
testY = LabelBinarizer().fit_transform(testY)

In [14]:
testY

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [15]:
opt = SGD(lr=0.05)
model = MiniVGGNet.build(width=64, height=64, depth=3,
                        classes=len(classNames))
model.compile(loss="categorical_crossentropy", optimizer=opt,
             metrics=["accuracy"])

In [16]:
H = model.fit(trainX, trainY, validation_data=(testX, testY),
             batch_size=32, epochs=100, verbose=1)

Train on 1020 samples, validate on 340 samples
Epoch 1/100
1020/1020 [==============================] - 22s - loss: 2.9538 - acc: 0.2814 - val_loss: 4.2174 - val_acc: 0.1618
Epoch 2/100
1020/1020 [==============================] - 1s - loss: 1.8698 - acc: 0.4657 - val_loss: 2.6445 - val_acc: 0.2000
Epoch 3/100
1020/1020 [==============================] - 1s - loss: 1.3153 - acc: 0.5706 - val_loss: 2.4931 - val_acc: 0.2147
Epoch 4/100
1020/1020 [==============================] - 1s - loss: 1.0316 - acc: 0.6637 - val_loss: 2.8167 - val_acc: 0.2029
Epoch 5/100
1020/1020 [==============================] - 1s - loss: 0.7176 - acc: 0.7578 - val_loss: 2.2765 - val_acc: 0.2941
Epoch 6/100
1020/1020 [==============================] - 1s - loss: 0.6698 - acc: 0.7686 - val_loss: 2.7520 - val_acc: 0.3176
Epoch 7/100
1020/1020 [==============================] - 1s - loss: 0.4957 - acc: 0.8294 - val_loss: 2.1872 - val_acc: 0.3676
Epoch 8/100
1020/1020 [==============================] - 1s - loss: 0.

1020/1020 [==============================] - 1s - loss: 0.0034 - acc: 1.0000 - val_loss: 1.6916 - val_acc: 0.6294
Epoch 66/100
1020/1020 [==============================] - 1s - loss: 0.0040 - acc: 1.0000 - val_loss: 1.6949 - val_acc: 0.6294
Epoch 67/100
1020/1020 [==============================] - 1s - loss: 0.0029 - acc: 1.0000 - val_loss: 1.6988 - val_acc: 0.6382
Epoch 68/100
1020/1020 [==============================] - 1s - loss: 0.0033 - acc: 1.0000 - val_loss: 1.7017 - val_acc: 0.6265
Epoch 69/100
1020/1020 [==============================] - 1s - loss: 0.0032 - acc: 1.0000 - val_loss: 1.7056 - val_acc: 0.6235
Epoch 70/100
1020/1020 [==============================] - 1s - loss: 0.0047 - acc: 0.9990 - val_loss: 1.7058 - val_acc: 0.6324
Epoch 71/100
1020/1020 [==============================] - 1s - loss: 0.0033 - acc: 1.0000 - val_loss: 1.7021 - val_acc: 0.6265
Epoch 72/100
1020/1020 [==============================] - 1s - loss: 0.0054 - acc: 0.9990 - val_loss: 1.7248 - val_acc: 0.60

In [17]:
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
                           predictions.argmax(axis=1), target_names=classNames))

              precision    recall  f1-score   support

    bluebell       0.67      0.90      0.77        20
   buttercup       0.50      0.60      0.55        20
   coltsFoot       0.74      0.52      0.61        27
     cowslip       0.45      0.57      0.50        23
      crocus       0.65      0.62      0.63        21
    daffodil       0.26      0.28      0.27        18
       daisy       0.63      0.80      0.71        15
   dandelion       0.71      0.67      0.69        18
  fritillary       0.75      0.63      0.69        19
        iris       0.95      0.82      0.88        22
  lilyvalley       0.52      0.69      0.59        16
       pansy       0.92      0.63      0.75        19
    snowdrop       0.56      0.56      0.56        16
   sunflower       0.86      0.83      0.84        23
   tigerlily       0.89      0.70      0.78        23
       tulip       0.50      0.60      0.55        20
  windflower       0.71      0.60      0.65        20

   micro avg       0.65   

In [18]:
from keras.preprocessing.image import ImageDataGenerator

In [20]:
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
                        height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                        horizontal_flip=True, fill_mode="nearest")

In [22]:
opt = SGD(lr=0.05)
model = MiniVGGNet.build(width=64, height=64, depth=3,
                        classes=len(classNames))
model.compile(loss="categorical_crossentropy", optimizer=opt,
             metrics=["accuracy"])

In [23]:
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=32),
                       validation_data=(testX, testY), steps_per_epoch=len(trainX) // 32,
                       epochs=100, verbose=1)

Epoch 1/100
31/31 [==============================] - 3s - loss: 3.4707 - acc: 0.1815 - val_loss: 2.5028 - val_acc: 0.2000
Epoch 2/100
31/31 [==============================] - 1s - loss: 2.6231 - acc: 0.2863 - val_loss: 2.4388 - val_acc: 0.2029
Epoch 3/100
31/31 [==============================] - 1s - loss: 2.2721 - acc: 0.3570 - val_loss: 3.0274 - val_acc: 0.1294
Epoch 4/100
31/31 [==============================] - 1s - loss: 1.9200 - acc: 0.4414 - val_loss: 3.1708 - val_acc: 0.1794
Epoch 5/100
31/31 [==============================] - 0s - loss: 1.7738 - acc: 0.4577 - val_loss: 2.7734 - val_acc: 0.2471
Epoch 6/100
31/31 [==============================] - 0s - loss: 1.7293 - acc: 0.4823 - val_loss: 2.5706 - val_acc: 0.2706
Epoch 7/100
31/31 [==============================] - 0s - loss: 1.5064 - acc: 0.5436 - val_loss: 1.7698 - val_acc: 0.4088
Epoch 8/100
31/31 [==============================] - 0s - loss: 1.4097 - acc: 0.5460 - val_loss: 1.7818 - val_acc: 0.4500
Epoch 9/100
31/31 [=====

31/31 [==============================] - 0s - loss: 0.4673 - acc: 0.8347 - val_loss: 1.1814 - val_acc: 0.7118
Epoch 68/100
31/31 [==============================] - 1s - loss: 0.3875 - acc: 0.8806 - val_loss: 1.1398 - val_acc: 0.7471
Epoch 69/100
31/31 [==============================] - 0s - loss: 0.3914 - acc: 0.8713 - val_loss: 1.2233 - val_acc: 0.6794
Epoch 70/100
31/31 [==============================] - 0s - loss: 0.3890 - acc: 0.8553 - val_loss: 1.0841 - val_acc: 0.7353
Epoch 71/100
31/31 [==============================] - 1s - loss: 0.3184 - acc: 0.8873 - val_loss: 1.0948 - val_acc: 0.7412
Epoch 72/100
31/31 [==============================] - 1s - loss: 0.3072 - acc: 0.8945 - val_loss: 1.1969 - val_acc: 0.7441
Epoch 73/100
31/31 [==============================] - 1s - loss: 0.3858 - acc: 0.8670 - val_loss: 1.2330 - val_acc: 0.7294
Epoch 74/100
31/31 [==============================] - 1s - loss: 0.3156 - acc: 0.8889 - val_loss: 1.1259 - val_acc: 0.7235
Epoch 75/100
31/31 [=========

In [27]:
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
     predictions.argmax(axis=1), target_names=classNames))

              precision    recall  f1-score   support

    bluebell       0.89      0.80      0.84        20
   buttercup       0.89      0.40      0.55        20
   coltsFoot       0.87      0.48      0.62        27
     cowslip       0.36      0.87      0.51        23
      crocus       0.86      0.57      0.69        21
    daffodil       0.41      0.50      0.45        18
       daisy       1.00      0.80      0.89        15
   dandelion       0.86      0.67      0.75        18
  fritillary       0.71      0.79      0.75        19
        iris       0.79      0.68      0.73        22
  lilyvalley       0.62      1.00      0.76        16
       pansy       0.92      0.63      0.75        19
    snowdrop       0.71      0.62      0.67        16
   sunflower       0.77      1.00      0.87        23
   tigerlily       0.95      0.87      0.91        23
       tulip       0.93      0.65      0.76        20
  windflower       0.82      0.90      0.86        20

   micro avg       0.72   